![Logo](https://raw.githubusercontent.com/BartaZoltan/deep-reinforcement-learning-course/main/notebooks/shared_assets/logo.png)

Made by **Zoltán Barta**


# 2. alkalom – Dinamikus programozási kísérletek

Ez a notebook kifejezetten az előző foglalkozáson implementált algoritmusokhoz kapcsolódó kísérletekhez készült. A cél a paraméterek érzékenységének vizsgálata és a különböző környezetek összehasonlítása. Az alapvető definíciókat az előző notebook tartalmazza, itt a hangsúly a gyakorlati vizsgálatokon van.


## Szükséges eszközök beimportálása

Töltsük be a szükséges függvényeket és definiáljuk a környezetet. Célszerű az előző notebookból importálható módon egy `gridworld_utils.py` modulba szervezni a kódot. Itt azonban közvetlenül bemásoljuk a függvényeket.


In [ ]:
import numpy as np
import random
from typing import Dict, Tuple, List

# A Gridworld, policy_evaluation, policy_iteration és value_iteration függvények legyenek elérhetők
# Ha külön modulban tárolnánk őket, importálhatók lennének. Itt ismét lemásoljuk őket.
class Gridworld:
    def __init__(self, rows: int, cols: int, terminals: List[Tuple[int, int]],
                 slip_prob: float = 0.0, default_reward: float = -1.0):
        self.rows = rows
        self.cols = cols
        self.terminals = terminals
        self.slip_prob = slip_prob
        self.default_reward = default_reward
        self.actions = ['U', 'D', 'L', 'R']

    def in_bounds(self, state):
        r, c = state
        return 0 <= r < self.rows and 0 <= c < self.cols

    def step(self, state, action):
        if state in self.terminals:
            return state, 0.0
        actual_action = action
        if self.slip_prob > 0 and random.random() < self.slip_prob:
            actual_action = random.choice([a for a in self.actions if a != action])
        r, c = state
        if actual_action == 'U':
            new_state = (max(r - 1, 0), c)
        elif actual_action == 'D':
            new_state = (min(r + 1, self.rows - 1), c)
        elif actual_action == 'L':
            new_state = (r, max(c - 1, 0))
        elif actual_action == 'R':
            new_state = (r, min(c + 1, self.cols - 1))
        else:
            new_state = state
        reward = 0.0 if new_state in self.terminals else self.default_reward
        return new_state, reward

def policy_evaluation(policy, env, gamma=1.0, theta=1e-4):
    V = { (r, c): 0.0 for r in range(env.rows) for c in range(env.cols) }
    for t in env.terminals:
        V[t] = 0.0
    while True:
        delta = 0
        for r in range(env.rows):
            for c in range(env.cols):
                state = (r, c)
                if state in env.terminals:
                    continue
                a = policy[state]
                next_state, reward = env.step(state, a)
                v_new = reward + gamma * V[next_state]
                delta = max(delta, abs(v_new - V[state]))
                V[state] = v_new
        if delta < theta:
            break
    return V

def policy_iteration(env, gamma=1.0, theta=1e-4):
    policy = {}
    for r in range(env.rows):
        for c in range(env.cols):
            if (r, c) not in env.terminals:
                policy[(r, c)] = random.choice(env.actions)
    iteration = 0
    while True:
        iteration += 1
        V = policy_evaluation(policy, env, gamma, theta)
        stable = True
        for r in range(env.rows):
            for c in range(env.cols):
                state = (r, c)
                if state in env.terminals:
                    continue
                old_action = policy[state]
                action_values = {}
                for a in env.actions:
                    next_state, reward = env.step(state, a)
                    action_values[a] = reward + gamma * V[next_state]
                best_action = max(action_values, key=action_values.get)
                policy[state] = best_action
                if best_action != old_action:
                    stable = False
        if stable:
            break
    return V, policy, iteration

def value_iteration(env, gamma=1.0, theta=1e-4):
    V = { (r, c): 0.0 for r in range(env.rows) for c in range(env.cols) }
    for t in env.terminals:
        V[t] = 0.0
    iteration = 0
    while True:
        iteration += 1
        delta = 0
        for r in range(env.rows):
            for c in range(env.cols):
                state = (r, c)
                if state in env.terminals:
                    continue
                action_values = []
                for a in env.actions:
                    next_state, reward = env.step(state, a)
                    action_values.append(reward + gamma * V[next_state])
                v_new = max(action_values)
                delta = max(delta, abs(v_new - V[state]))
                V[state] = v_new
        if delta < theta:
            break
    policy = {}
    for r in range(env.rows):
        for c in range(env.cols):
            state = (r, c)
            if state in env.terminals:
                continue
            action_values = {}
            for a in env.actions:
                next_state, reward = env.step(state, a)
                action_values[a] = reward + gamma * V[next_state]
            policy[state] = max(action_values, key=action_values.get)
    return V, policy, iteration


## 1. γ és θ paraméterek részletes vizsgálata

Ebben a szakaszban részletes hőtérképeket vagy grafikonokat készítünk a konvergencia sebességéről a diszkontráta és a konvergencia-küszöb függvényében. A bemeneti paraméterek listáit tetszés szerint bővíthetjük.


In [ ]:
import pandas as pd
import itertools

def parameter_sweep(gammas, thetas, rows=4, cols=4):
    data = []
    terminal_states = [(rows - 1, cols - 1)]
    for gamma, theta in itertools.product(gammas, thetas):
        env = Gridworld(rows, cols, terminal_states)
        _, _, it_pi = policy_iteration(env, gamma=gamma, theta=theta)
        _, _, it_vi = value_iteration(env, gamma=gamma, theta=theta)
        data.append({'gamma': gamma, 'theta': theta, 'policy_iter': it_pi, 'value_iter': it_vi})
    df = pd.DataFrame(data)
    return df

# Példa paraméterkészlet
gammas = [0.5, 0.7, 0.9, 0.99]
thetas = [1e-1, 1e-2, 1e-3, 1e-4]

df = parameter_sweep(gammas, thetas)
df


Az így kapott DataFrame-ben jól látható, hogy a diszkontráta növelése általában több iterációt igényel. A konvergencia-küszöb csökkentése (például 1e-4 helyett 1e-1) jelentősen csökkenti az iterációk számát, de pontatlanabb értékfüggvényt eredményez. Használjunk `df.pivot_table` segítségével hőtérképet vagy grafikont is az adatok vizualizálására.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Hőtérkép készítése a value iteration iterációszámára
pivot_vi = df.pivot_table(values='value_iter', index='gamma', columns='theta')
plt.figure(figsize=(6, 4))
sns.heatmap(pivot_vi, annot=True, fmt='d', cmap='YlGnBu')
plt.title('Értékiteráció iterációszám')
plt.ylabel('gamma')
plt.xlabel('theta')
plt.show()

# Hőtérkép készítése a policy iteration iterációszámára
pivot_pi = df.pivot_table(values='policy_iter', index='gamma', columns='theta')
plt.figure(figsize=(6, 4))
sns.heatmap(pivot_pi, annot=True, fmt='d', cmap='YlOrBr')
plt.title('Policy iteráció iterációszám')
plt.ylabel('gamma')
plt.xlabel('theta')
plt.show()


## 2. Nagyobb rácsok

Készítsünk 5×5 és 6×6 méretű rácsokat, és vizsgáljuk meg, hogyan nő az iterációk száma. A fenti függvényekkel könnyen általánosíthatjuk a vizsgálatot, de a nagyobb állapottér miatt az algoritmusok lassulni fognak.


In [ ]:
# Nagyobb rács vizsgálata 5x5 és 6x6 méretben
sizes = [(5, 5), (6, 6)]
for size in sizes:
    rows, cols = size
    terminal_states = [(rows - 1, cols - 1)]
    env = Gridworld(rows, cols, terminal_states)
    _, _, it_pi = policy_iteration(env, gamma=0.9, theta=1e-3)
    _, _, it_vi = value_iteration(env, gamma=0.9, theta=1e-3)
    print(f'{rows}x{cols} rács: Policy iteráció {it_pi} iteráció, Értékiteráció {it_vi} iteráció')


## 3. Stochasztikus környezet és csúszás

A `slip_prob` paraméter beállításával sztochasztikus átmeneteket hozhatunk létre. Ilyenkor a step függvény különböző kimeneteket adhat, ezért az értékek kiszámításakor a várható értéket kell figyelembe venni. Egyszerűsített tanulmányozásához a slip valószínűséget 0,1 vagy 0,2 értékre állítjuk.


In [ ]:
def value_iteration_stochastic(env: Gridworld, gamma=1.0, theta=1e-4):
    V = { (r, c): 0.0 for r in range(env.rows) for c in range(env.cols) }
    for t in env.terminals:
        V[t] = 0.0
    iteration = 0
    while True:
        iteration += 1
        delta = 0
        for r in range(env.rows):
            for c in range(env.cols):
                state = (r, c)
                if state in env.terminals:
                    continue
                action_values = []
                for a in env.actions:
                    total = 0.0
                    # slip_prob szerint súlyozott várható érték
                    for possible_action in env.actions:
                        prob = env.slip_prob / (len(env.actions) - 1) if possible_action != a else 1 - env.slip_prob
                        next_state, reward = env.step(state, possible_action)
                        total += prob * (reward + gamma * V[next_state])
                    action_values.append(total)
                v_new = max(action_values)
                delta = max(delta, abs(v_new - V[state]))
                V[state] = v_new
        if delta < theta:
            break
    # Politika származtatása
    policy = {}
    for r in range(env.rows):
        for c in range(env.cols):
            state = (r, c)
            if state in env.terminals:
                continue
            action_values = {}
            for a in env.actions:
                total = 0.0
                for possible_action in env.actions:
                    prob = env.slip_prob / (len(env.actions) - 1) if possible_action != a else 1 - env.slip_prob
                    next_state, reward = env.step(state, possible_action)
                    total += prob * (reward + gamma * V[next_state])
                action_values[a] = total
            policy[state] = max(action_values, key=action_values.get)
    return V, policy, iteration


In [ ]:
# Példa: sztochasztikus 4x4 környezet slip_prob=0.2
env_stochastic = Gridworld(4, 4, [(3, 3)], slip_prob=0.2)
V_stoch, policy_stoch, it_stoch = value_iteration_stochastic(env_stochastic, gamma=0.9, theta=1e-3)
print(f'Sztochasztikus értékiteráció {it_stoch} iteráció alatt konvergált.')
print('Optimális politika sztochasztikus környezetben:')
print('
'.join([' '.join([policy_stoch.get((r, c), 'T') for c in range(4)]) for r in range(4)]))


## 4. FrozenLake-szerű környezet létrehozása

**Feladat:** Implementálják a fenti Gridworld osztály speciális példányát, ahol néhány állapot lyukként (negatív jutalom, azonnali terminál) viselkedik, és a célállapot pozitív jutalmat ad. Ezt követően futtassák rajta a value iteration vagy policy iteration algoritmust.


## 5. Összefoglaló

Ezek a kísérletek segítenek abban, hogy mélyebben megértsük, hogyan hatnak a hiperparaméterek a dinamikus programozási algoritmusokra és hogyan viselkednek ezek különböző környezetekben. Vizsgáljuk meg a változásokat vizuálisan is, és beszéljük meg a tanulságokat.
